<a href="https://colab.research.google.com/github/DajeanArcila/biblioteca_pandas/blob/main/Redes_Neurais_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# Carregando base de dados

df = pd.read_excel("churn.xlsx")

In [3]:
df.head()

customerID  tenure PhoneService        Contract PaperlessBilling  \
0  7590-VHVEG       1           No  Month-to-month              Yes   
1  5575-GNVDE      34          Yes        One year               No   
2  3668-QPYBK       2          Yes  Month-to-month              Yes   
3  7795-CFOCW      45           No        One year               No   
4  9237-HQITU       2          Yes  Month-to-month              Yes   

      PaymentMethod  MonthlyCharges  TotalCharges Churn  
0  Electronic check           29.85         29.85    No  
1      Mailed check           56.95       1889.50    No  
2      Mailed check           53.85        108.15   Yes  
3     Bank transfer           42.30       1840.75    No  
4  Electronic check           70.70        151.65   Yes

In [4]:
#Excluindo variáveis não úteis

df = df.drop(columns=["customerID"])

In [5]:
df = df.dropna()

In [7]:
df.isnull().any()

tenure              False
PhoneService        False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
Churn               False
dtype: bool

In [8]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors='coerce')

In [9]:
cat_vars = ['PhoneService',"Contract","PaperlessBilling","PaymentMethod"]
num_vars = ["tenure","MonthlyCharges","TotalCharges"]
modeling_vars = cat_vars + num_vars

In [10]:
X = df.filter(modeling_vars).copy()
y = df['Churn'].copy()
y = pd.DataFrame(y)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(y.Churn)
y.Churn = le.transform(y.Churn)

In [12]:
X = pd.get_dummies(X)

In [13]:
# Distruibuição da variáveis target
df["Churn"].value_counts()


Churn
No     5162
Yes    1869
Name: count, dtype: int64

In [14]:
# Dividindo a base em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test: {X_test.shape}")
print(f"Shape y_test: {y_test.shape}")

Shape X_train: (5624, 14)
Shape y_train: (5624, 1)
Shape X_test: (1407, 14)
Shape y_test: (1407, 1)


In [15]:
# Normalização das variáveis

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Transformanda para dataframe para visualização
X_train = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_test = pd.DataFrame(X_test_scaled,columns = X_test.columns)

In [18]:
# Classificador Multi Layer Perceptron

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

clf = MLPClassifier(hidden_layer_sizes=(100, 50, 20),activation='logistic')
clf.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50, 20))

In [20]:
y_pred = clf.predict(X_test)
print(f"Acurácia: {accuracy_score(y_test, y_pred)*100}%")

Acurácia: 79.67306325515281%


In [22]:
print(f"F1 Score: {f1_score(y_test, y_pred)}")

F1 Score: 0.5545171339563864


In [23]:
#f1 score de 55 porciento